In [7]:
# 取得的字段包括：日期，标题，url，内容描述，发布者，发布主页，发布者是否为机构号，发布时间，进入热榜时间，热度，回复数，话题类型
import json
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup

In [8]:
# 获取文件名（热榜日期）
filePath = r'X:\pyProjs\zhihu-top-search\raw'
for i,j,k in os.walk(filePath):
    fileDate = []
    fileNames = k
    for item in k:
        fileDate.append(item[:-5])

In [9]:
def getDetailInfo(url):
    url = url + r'/log'
    print(url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        'Origin': 'https://www.zhihu.com',
        'Cookie': 'SESSIONID=z8fQBb4qJMmEobRDGXFdXrTXRWQPVvEkvaQqLluO3VU; osd=UFEVBUnIbRZlAVlLIsxGTfPMg9Ex4EczQiJxYQfrRTxAJnpjCHGMegAKW0Qg8uVkzNSaqPJwZhK07WQGxGqU-lo=; JOID=V1gUBEPPZBdkC15CI81MSvrNgts26UYySCV4YAbhQjVBJ3BkAXCNcAcDWkUq9exlzd6dofNxbBW97GUMw2OV-1A=; _zap=7c6b1178-7ecf-4461-8102-605774e55301; d_c0="AICSY5B5gxGPTrOMPR5UTZEJEa6iT3yPENQ=|1593660904"; _ga=GA1.2.1934862013.1593660905; _xsrf=ffe04MllUDtjKNWzNio25uYOXrASgbsq; __snaker__id=tAeJ0FlSoB76vsVm; _9755xjdesxxd_=32; YD00517437729195%3AWM_TID=nMOwYnz4Wx1BUQVRVEJ%2F1MBvS%2BaVFbM8; __utmv=51854390.100--|2=registration_date=20210317=1^3=entry_date=20200709=1; __utmz=51854390.1626961624.2.2.utmcsr=zhihu.com|utmccn=(referral)|utmcmd=referral|utmcct=/question/473802687; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1634351793,1634561752,1634563375,1634660025; gdxidpyhxdE=yI8wbeZ%5CHpQGXIQ%2BguZ%5CCRgqa5dRop%5CLS3HsQNmUpeZk9hd8YxAHTS3bzmKSXMwx3PB9tXZJ%2FViYLs1mvumf%2B1yi9pmNqNvjwadnXItJucSaxl9aldBDRcC3AoeN4wB%2FbN8neBDibevqdIi1RZYlEoKZb2bG6KqHhXPTUneGu50bGsqL%3A1634660927156; YD00517437729195%3AWM_NI=c90cP4Q2oGizDeBphXp%2BeXehv0y%2B2S9SNiiZqrmBvmncbVdnkNYQr9D%2BaNiNZFLOp0lCbA2WpIVdwiSMjRxRrxlJOvIrK5ZKhuHHcZRQvVBgK2gxbGVPdaQEuV5FaLgaNE4%3D; YD00517437729195%3AWM_NIKE=9ca17ae2e6ffcda170e2e6eea2c460a6ae8fa4e65a8ae78ab3c14e838f8e84f4799bba9daeec5083f18f85d12af0fea7c3b92aa1e70084d34093eea1a8ec7288bb8e91eb54b4f5abb9e864b6f0b7ccb159fb8d828aec52b3aea3a3f567aebdaeb7e766a99297b2f13388f5afafec469a9a84b0f33cb09daea2b15db4979a85f6639193a6b3fb5ab2a8a4abce7e8bae85d6d941acab88a9e252928fbab7c16d89eea999e14d9690ac82d44fb78afba7e448a5bf82a6b737e2a3; l_n_c=1; l_cap_id="N2M5ZjIxOGI0MGVlNGRlMzgyMDUzM2RiMzcwYjkxN2U=|1634660049|987bf4c70953a7ef070519d816b97bf2d232a512"; r_cap_id="ODdlMTgwMDI2ZjQ5NGUyOWE2MjgxYzAyZWJjZTc1NjQ=|1634660049|017dca5083291d30771993d1b47a7eb8fc0e12ae"; cap_id="NGU4MmZkNTFlY2NlNDU0ZTgzNjQyNzUyZjFmMzg4Y2Q=|1634660049|521fb82a5206009b77722e9429ccc333a09e38f5"; n_c=1; captcha_session_v2="2|1:0|10:1634660114|18:captcha_session_v2|88:SUp2bTZZTjM3dHFDRERJQmFBWC9aemJ6WXhkSjBCL05VUG5WQlpld09wdk0zTzRSdG1CczlhWTloVDRTQUV1Mw==|3533890e2d2c98b2603d05d00ca5ac6ee12b7bd0ea0cfc5795beb03b837d1270"; captcha_ticket_v2="2|1:0|10:1634660122|17:captcha_ticket_v2|704:eyJ2YWxpZGF0ZSI6IkNOMzFfMHZSLjV4cFhKZ2RhMkNJY0JoMDhma2pLZEs3YXljTDgyb1BtSy11SF9aSmFhN3lpT1R6WGlMbUVqQ1RWQUNjX2JHdzdMbmg4eDB0YWRKRWk2LWJTWnlmZ1pXVXRsOGJmR20uZ3ROVkh3U0lYWnZBMW80cGZDdE5LUmdSdnF5RjJzZmljb1BkbVJ4aVN6WHRuc256MEVCbUUwUVdqZ3d5cGVXeFkuLU1aLlJCQzFIRGtzaTBVcmp1aVk0cS5LbWNDaEQ1QTB5R2ZialhlZmxwRXpMaFBXd25tZUU2NGh2Q24wMDJsMmFfd0ZfTElJb3ZkN0FJTjBidkJaUVJyZlBnb0VOVlRrLU92LVRxNFRCc3RqeVZNYUFqcGEudnNTcndlcmpjSFk0al9MS19NRDFtMVJDXzFXWFphNERsc1lZVVNiRnlqdW1KdFFIcVlsUkxIXy1ra1Y3cTJVRGhKSEZxMGx1a1lNcHZldVcuMHRJc2xqNXNwRC1PNXVvVnd5cERieWVXaHpnUGJ0NHNaNkNKaXJJZC0tcGcwZHc5RVdKcDVUeWs5Zng5QXlINjUxR21DNWtuWi1rRDhCUlUtMEV6c2JSTW9JY1N3SjcySEZoLkVmbjI3RnRnQnAuVXFWaGJ5eHJjY2JyUHhYLm9NS3FULkZWX3l6RC1GeTBaMyJ9|6f750776e2778fb05170e69ba09def54c9f66701cd6c7aa7e105b164e67a8c0d"; z_c0="2|1:0|10:1634660139|4:z_c0|92:Mi4xZWlTakt3QUFBQUFBZ0pKamtIbURFU1lBQUFCZ0FsVk5LejFjWWdBNkJ3Q0piUE4xTks4Q2Z4MHg2bEoyY0s4a0NB|f65065f1a250e2cfc49e57d444ac431c50107835a3ca3cc46307691af77e2810"; unlock_ticket="AGCcbsUf0BImAAAAYAJVTTP2bmFGfXxcgiX2Y2EYN5uY1cWNqAeZFw=="; Hm_lpvt_98beee57fd2ef70ccdd5ca52b9740c49=1634660261; q_c1=ad266df5c57c402098b8dbc61971e450|1634660266000|1594278698000; KLBRSID=fe78dd346df712f9c4f126150949b853|1634660266|1634658106; __utma=51854390.1934862013.1593660905.1627290591.1634660267.7; __utmb=51854390.0.10.1634660267; __utmc=51854390'
    }
    try:
        res = requests.get(url, headers=headers)
    except:
        return [None, None, None, None, None]
    else:
        soup = BeautifulSoup(res.text, 'lxml')
        try:
            time = soup.find_all('time')
            created_date = time[-1].string
            hot_date = time[0].string
        except:
            created_date = ''
            hot_date = ''
        try:
            author = soup.find_all(class_='zm-item')[-1].select('a')[0].string
            author_url = 'https://www.zhihu.com' + soup.find_all(class_='zm-item')[-1].select('a')[0].attrs['href']
        except:
            author = '匿名'
            author_url = '匿名无主页'
        content = ''
        try:
            for p in soup.find_all(class_='zm-item')[-1].select('ins')[1].select('p'):
                content += p.string
        except:
            content = ''
        return [created_date, hot_date, author, author_url, content]
    

In [10]:
num = 0
outArr = []
for file in fileNames:
    path = os.path.join(r'X:\pyProjs\zhihu-top-search\raw', file)
    date = file[:-5] # 输出字段
    rank = 0
    with open(path, 'r', encoding="utf-8") as f:
        textDict = json.load(f)
        print(textDict)
        print(type(textDict))
        for i, k in textDict.items():
            print(k)
            title = k['title']
            url = k['url']
            hot = int(k['hot'][:-4])
            answerCount = k['answerCount']
            rank += 1
            created_date, hot_date, author, author_url, content = getDetailInfo(url)
            row = [date, title, url, hot, answerCount, rank, created_date, hot_date, author, author_url, content]
            print(row)
            outArr.append(row)
            if rank >=10:
                break
    num += 1
    if num >=60:
        break

{'457140816': {'title': '国家统计局称 2020 年我国人口继续保持增长，你怎么看？还有哪些信息值得关注？', 'url': 'https://www.zhihu.com/question/457140816', 'hot': '4778 万热度', 'answerCount': 1387}, '457181415': {'title': '75 位一线艺人注销 200 家关联公司，如何看待众艺人集体注销关联公司？', 'url': 'https://www.zhihu.com/question/457181415', 'hot': '2058 万热度', 'answerCount': 81}, '456399195': {'title': '印度的这次疫情大爆发，是否已经发生了 ADE 效应？', 'url': 'https://www.zhihu.com/question/456399195', 'hot': '1940 万热度', 'answerCount': 24}, '456846832': {'title': '如何看待董明珠给新员工分房，称多给一万块没用，房子才能解决根本问题？', 'url': 'https://www.zhihu.com/question/456846832', 'hot': '1866 万热度', 'answerCount': 632}, '457029348': {'title': '如何看待郑爽涉嫌阴阳合同、天价片酬、偷逃税被调查？可能会带来哪些影响？', 'url': 'https://www.zhihu.com/question/457029348', 'hot': '1769 万热度', 'answerCount': 2606}, '444218811': {'title': '1+0.1+0.01+0.001+0.0001... 一直下去会在实际中到达 2 吗？', 'url': 'https://www.zhihu.com/question/444218811', 'hot': '1675 万热度', 'answerCount': 1645}, '457141906': {'title': '如何看待赵薇被冻结数百万股权，发生了什么？可能会有哪些影响？', 'url': 'https://ww

['20210430', '国家统计局称 2020 年我国人口继续保持增长，你怎么看？还有哪些信息值得关注？', 'https://www.zhihu.com/question/457140816', 4778, 1387, 1, '2021-04-29 14:38:36', '2021-04-29 15:59:00', '中国新闻网', 'https://www.zhihu.com/people/zhong-guo-xin-wen-wang-93', '']
{'title': '75 位一线艺人注销 200 家关联公司，如何看待众艺人集体注销关联公司？', 'url': 'https://www.zhihu.com/question/457181415', 'hot': '2058 万热度', 'answerCount': 81}
https://www.zhihu.com/question/457181415/log
['20210430', '75 位一线艺人注销 200 家关联公司，如何看待众艺人集体注销关联公司？', 'https://www.zhihu.com/question/457181415', 2058, 81, 2, '2021-04-29 19:53:05', '2021-04-30 09:42:11', '36氪', 'https://www.zhihu.com/people/36-ke', '29日，郑爽涉嫌偷逃税涉事企业被约谈。郑爽、黄晓明、何炅、黄圣依、唐嫣、赵薇等75位一线艺人相关公司发现，75位艺人名下相关公司共有647家，其中已注销200家。此外，艺人名下关联公司数量最多的是黄晓明，其相关公司一共有43家，其次为章子怡，名下相关公司有29家，为女艺人中公司数量最多。此外，陈坤有25家，李晨有21家，黄磊有20家，何炅有19家，陈赫18家，靳东18家，郑恺17家，黄渤16家，赵薇14家。（山东商报）']
{'title': '印度的这次疫情大爆发，是否已经发生了 ADE 效应？', 'url': 'https://www.zhihu.com/question/456399195', 'hot': '1940 万热度', 'answerCount': 24}
https://www.zhihu.com/question/45639

['20210501', '如何评价电影《你的婚礼》？', 'https://www.zhihu.com/question/437513111', 2534, 270, 1, '2021-01-02 11:25:17', '2021-04-30 19:22:36', '文中有蓝', 'https://www.zhihu.com/people/sunnyamily', '']
{'title': '如何看待腾讯起诉 DD373，庭审称「虚拟财产不属于用户，被盗用户需赔偿」？是否有法律依据？', 'url': 'https://www.zhihu.com/question/457298163', 'hot': '2386 万热度', 'answerCount': 34}
https://www.zhihu.com/question/457298163/log
['20210501', '如何看待腾讯起诉 DD373，庭审称「虚拟财产不属于用户，被盗用户需赔偿」？是否有法律依据？', 'https://www.zhihu.com/question/457298163', 2386, 34, 2, '2021-04-30 17:19:20', '2021-05-02 20:03:44', '匿名', '匿名无主页', '']
{'title': '如何看待印度拒绝联合国物资驰援，因印度有「健全的体系」来处理问题？', 'url': 'https://www.zhihu.com/question/457285008', 'hot': '1977 万热度', 'answerCount': 367}
https://www.zhihu.com/question/457285008/log
['20210501', '如何看待印度拒绝联合国物资驰援，因印度有「健全的体系」来处理问题？', 'https://www.zhihu.com/question/457285008', 1977, 367, 3, '2021-04-30 15:30:49', '2021-04-30 19:01:40', '观察者网', 'https://www.zhihu.com/people/guan-cha-zhe-wang-31', ' \t据《印度报》29日消息，联合国秘书长副发言人哈克（Farhan

['20210502', '如何看待腾讯起诉游戏交易平台，庭审称「虚拟财产不属于用户，被盗用户需赔偿」？', 'https://www.zhihu.com/question/457298163', 2347, 908, 1, '2021-04-30 17:19:20', '2021-05-02 20:03:44', '匿名', '匿名无主页', '']
{'title': '如何看待全球新冠确诊超 1.5 亿，印度单日新增确诊连续 9 天超 30 万例，未来国际疫情形势如何？', 'url': 'https://www.zhihu.com/question/457368252', 'hot': '1723 万热度', 'answerCount': 170}
https://www.zhihu.com/question/457368252/log
['20210502', '如何看待全球新冠确诊超 1.5 亿，印度单日新增确诊连续 9 天超 30 万例，未来国际疫情形势如何？', 'https://www.zhihu.com/question/457368252', 1723, 170, 2, '2021-05-01 08:49:14', '2021-05-01 11:03:07', '中国新闻网', 'https://www.zhihu.com/people/zhong-guo-xin-wen-wang-93', '世界卫生组织30日公布的最新数据显示，全球累计新冠确诊病例达150110310例。印度卫生部30日公布的数据显示，该国较前一日新增新冠确诊病例386452例，累计18762976例；新增死亡3498例，累计208330例。印度单日新增确诊病例已连续9天超过30万例。']
{'title': '单日新增逾 40 万例新冠肺炎确诊病例，印度这是咋了？', 'url': 'https://www.zhihu.com/question/457388433', 'hot': '1609 万热度', 'answerCount': 28}
https://www.zhihu.com/question/457388433/log
['20210502', '单日新增逾 40 万例新冠肺炎确诊病例，印度这是咋了？', 'https://www.zhihu.com/ques

['20210503', '如何看待张国立发布大夜班工作动态，被网友指责「明星赚得多还矫情」这件事？', 'https://www.zhihu.com/question/457625710', 3908, 140, 1, '2021-05-03 10:30:59', '2021-05-03 13:26:46', '东南天x', 'https://www.zhihu.com/people/xmf-70', '']
{'title': '如何看待《青春有你 3》牛奶打投，粉丝为投票将大量未饮用的牛奶倒入下水道？', 'url': 'https://www.zhihu.com/question/457119531', 'hot': '2474 万热度', 'answerCount': 686}
https://www.zhihu.com/question/457119531/log
['20210503', '如何看待《青春有你 3》牛奶打投，粉丝为投票将大量未饮用的牛奶倒入下水道？', 'https://www.zhihu.com/question/457119531', 2474, 686, 2, '2021-04-29 11:43:18', '2021-05-02 16:13:41', '白月初', 'https://www.zhihu.com/people/bai-yue-chu-48-86', '']
{'title': '单日新增逾 40 万例新冠肺炎确诊病例，印度这是咋了？', 'url': 'https://www.zhihu.com/question/457388433', 'hot': '1939 万热度', 'answerCount': 313}
https://www.zhihu.com/question/457388433/log
['20210503', '单日新增逾 40 万例新冠肺炎确诊病例，印度这是咋了？', 'https://www.zhihu.com/question/457388433', 1939, 313, 3, '2021-05-01 12:32:33', '2021-05-07 17:30:16', '中国新闻网', 'https://www.zhihu.com/people/zhong-guo-xin-wen-wang-9

...

['20210527', '雄安被确认为婚俗改革实验区，通过 3 年实验遏制婚俗不正之风，或将产生哪些具体影响？', 'https://www.zhihu.com/question/461486744', 4135, 538, 1, '2021-05-26 13:20:25', '2021-05-26 16:27:33', '中新社国是直通车', 'https://www.zhihu.com/people/zhong-xin-she-guo-shi-zhi-tong-che', '据河北省民政厅26日消息，雄安新区等近日被确认为全省婚俗改革实验区，将大力推进婚姻领域移风易俗。并要求通过3年实验，实验区内高价彩礼、人情攀比等不良风气将得到有效遏制。']
{'title': '如何评价美国 2021 年的征兵广告？', 'url': 'https://www.zhihu.com/question/461629217', 'hot': '3318 万热度', 'answerCount': 38}
https://www.zhihu.com/question/461629217/log
['20210527', '如何评价美国 2021 年的征兵广告？', 'https://www.zhihu.com/question/461629217', 3318, 38, 2, '2021-05-27 09:46:17', '2021-05-27 14:35:33', '观察者网', 'https://www.zhihu.com/people/guan-cha-zhe-wang-31', '']
{'title': '如果恶魔给你一亿，一年之后必须还两个亿，你会接受吗？', 'url': 'https://www.zhihu.com/question/392418796', 'hot': '3257 万热度', 'answerCount': 454}
https://www.zhihu.com/question/392418796/log
['20210527', '如果恶魔给你一亿，一年之后必须还两个亿，你会接受吗？', 'https://www.zhihu.com/question/392418796', 3257, 454, 3, '2020-05-03 22:30:21', 

['20210528', '如果恶魔给你一亿，一年之后必须还两个亿，你会接受吗？', 'https://www.zhihu.com/question/392418796', 3316, 793, 1, '2020-05-03 22:30:21', '2021-05-27 17:27:39', 'operation', 'https://www.zhihu.com/people/operation-24', 'PS：不能在知乎宇宙中刷钱 ']
{'title': '如何评价美国 2021 年的征兵广告？', 'url': 'https://www.zhihu.com/question/461629217', 'hot': '3237 万热度', 'answerCount': 836}
https://www.zhihu.com/question/461629217/log
['20210528', '如何评价美国 2021 年的征兵广告？', 'https://www.zhihu.com/question/461629217', 3237, 836, 2, '2021-05-27 09:46:17', '2021-05-27 14:35:33', '观察者网', 'https://www.zhihu.com/people/guan-cha-zhe-wang-31', '']
{'title': '台大医院院长发文求救，目前台湾疫情情况如何？主要困难有哪些？', 'url': 'https://www.zhihu.com/question/461718906', 'hot': '2935 万热度', 'answerCount': 532}
https://www.zhihu.com/question/461718906/log
['20210528', '台大医院院长发文求救，目前台湾疫情情况如何？主要困难有哪些？', 'https://www.zhihu.com/question/461718906', 2935, 532, 3, '2021-05-27 18:27:09', '2021-05-28 10:43:43', '中国新闻网', 'https://www.zhihu.com/people/zhong-guo-xin-wen-wang-93', '\u3000

['20210529', '中国「人造太阳」EAST 创世界纪录实现 1 亿摄氏度百秒，有哪些意义？我们离可控核聚变更近了吗？', 'https://www.zhihu.com/question/461890685', 2663, 52, 1, '2021-05-28 17:18:54', '2021-05-28 23:12:47', 'AlbertAndVivian', 'https://www.zhihu.com/people/workalbertx', '']
{'title': '台大医院院长发文求救，目前台湾疫情情况如何？主要困难有哪些？', 'url': 'https://www.zhihu.com/question/461718906', 'hot': '2650 万热度', 'answerCount': 627}
https://www.zhihu.com/question/461718906/log
['20210529', '台大医院院长发文求救，目前台湾疫情情况如何？主要困难有哪些？', 'https://www.zhihu.com/question/461718906', 2650, 627, 2, '2021-05-27 18:27:09', '2021-05-28 10:43:43', '中国新闻网', 'https://www.zhihu.com/people/zhong-guo-xin-wen-wang-93', '\u3000台湾疫情近日持续升温，尤其以双北最为严峻。台大医院院长吴明贤5月26日通过社交平台向社会发出“求救信”，描述了医院病患爆满、病房满床的现状。\u3000\u3000综合“中央社”、中评社消息，吴明贤以“医院需要帮助”(Hospitals Need Help)为题发文指出，医院现在非常紧急，台大改装儿童加护病房都来收治成人，仍然不敷使用，而救护车持续把需要加护病房和插管的确诊病人送来。吴明贤呼吁台疫情中心重视这批大量重症病患，并称这样下去会很惨。\u3000\u3000吴明贤指出，台大医院已收治超过100名新冠个案，重症有20多人，是全台最多，已经满载，抗疫如作战，由于新冠病患人数实在太多，若没有准备完善，这样下去后果可想而知。他呼吁台当局统筹考量，给出办法。\u3000\u30005月25日，台北市立联合医院企

['20210530', '南京警方通报 5 月 29 日一嫌疑男子驾车撞人并持刀捅人，嫌疑人已被抓获，有哪些信息值得关注？', 'https://www.zhihu.com/question/462129219', 7766, 321, 1, '2021-05-30 00:26:34', '2021-05-30 13:49:05', '经济观察报', 'https://www.zhihu.com/people/jing-ji-guan-cha-bao', '']
{'title': '加拿大一印第安寄宿学校旧址发现 215 具孩童遗骸，这是怎么回事？', 'url': 'https://www.zhihu.com/question/462022143', 'hot': '2770 万热度', 'answerCount': 75}
https://www.zhihu.com/question/462022143/log
['20210530', '加拿大一印第安寄宿学校旧址发现 215 具孩童遗骸，这是怎么回事？', 'https://www.zhihu.com/question/462022143', 2770, 75, 2, '2021-05-29 13:36:16', '2021-05-30 11:34:32', '匿名', '匿名无主页', '']
{'title': '高三作弊，父母以为我成绩很好，现在该高考了，该怎么办?', 'url': 'https://www.zhihu.com/question/461546823', 'hot': '1756 万热度', 'answerCount': 714}
https://www.zhihu.com/question/461546823/log
['20210530', '高三作弊，父母以为我成绩很好，现在该高考了，该怎么办?', 'https://www.zhihu.com/question/461546823', 1756, 714, 3, '2021-05-26 19:26:31', '2021-05-29 18:14:22', '匿名', '匿名无主页', '我现在知道学了，但是以前落下太多了，真的特别后悔，我明年想复读，但是我爸妈以为我成绩很好，我该怎么办？']
{'title': '如何看待女子网

['20210531', '如何看待我国将进一步优化生育政策，一对夫妻可以生育三个子女？', 'https://www.zhihu.com/question/462390587', 13421, 4228, 1, '2021-05-31 15:02:48', '2021-05-31 15:39:32', '中国新闻网', 'https://www.zhihu.com/people/zhong-guo-xin-wen-wang-93', '积极应对人口老龄化，我国将出台重大政策举措。中共中央政治局5月31日召开会议，会议指出，进一步优化生育政策，实施一对夫妻可以生育三个子女政策及配套支持措施，有利于改善我国人口结构、落实积极应对人口老龄化国家战略、保持我国人力资源禀赋优势。（新华社） \u200b']
{'title': '北大数学天才韦东奕手拿馒头矿泉水受访，他在数学领域取得了哪些成就？给我们哪些启示？', 'url': 'https://www.zhihu.com/question/462169322', 'hot': '5538 万热度', 'answerCount': 788}
https://www.zhihu.com/question/462169322/log
['20210531', '北大数学天才韦东奕手拿馒头矿泉水受访，他在数学领域取得了哪些成就？给我们哪些启示？', 'https://www.zhihu.com/question/462169322', 5538, 788, 2, '2021-05-30 10:13:41', '2021-06-03 10:31:43', '无名泽雪', 'https://www.zhihu.com/people/nemotv', '']
{'title': '南京警方通报 5 月 29 日一嫌疑男子驾车撞人并持刀捅人，嫌疑人已被抓获，有哪些信息值得关注？', 'url': 'https://www.zhihu.com/question/462129219', 'hot': '4022 万热度', 'answerCount': 1368}
https://www.zhihu.com/question/462129219/log
['20210531', '南京警方通报 5 月 29 日一嫌疑男子驾车撞人并持刀捅人，

In [11]:
df = pd.DataFrame(columns=['热榜日期','标题', '问题主页','热度', '回答数量', '排名', '问题创建时间', '进入热榜时间', '提问者', '提问者主页', '问题内容描述'])
for i in range(len(outArr)):
    df.loc[i] = outArr[i]

In [12]:
df.to_excel('./test.xlsx')